In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore") 
import sys
sys.path.append('../scripts/')
from metrics import find_metrics, find_metrics_macro

/home/paperspace/Documents/tensor_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../ml_research_data/dolphin_dataset/files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../ml_research_data/dolphin_dataset/files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder, image_size, return_label = True):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        self.return_label = return_label
        self.image_size = image_size

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (self.image_size[0], self.image_size[1]))
        img = img / 255.0
        if self.return_label:
            return img, torch.Tensor(self.encoded_labels[idx])
        else:
            return img

In [8]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [9]:
def get_model(load_weights = None):
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    if load_weights != None:
        incep_model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    incep_model.eval()
    return incep_model

In [10]:
categorical_cross_entropy = nn.CrossEntropyLoss()

In [11]:
#  prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
# Appply batch size as well

In [24]:
def itereate_dataset(dataset, model):
    loss_, prec_, rec_ = 0.0, 0.0, 0.0
    for img, labels in tqdm(dataset):
        img = img.to(device)
        labels = labels.to(device)
        outputs = model(img)
        loss = categorical_cross_entropy(outputs, labels)
        prec, rec = find_metrics_macro(outputs, labels, batch_size=64)
        loss_ += loss.item()
        prec_ += prec.item()
        rec_ += rec.item()
    return loss_, prec_, rec_
    

In [34]:
def eval_model(model, train_dataset=None, val_dataset=None):
    model = model.to(device)
    if train_dataset != None:
        loss, prec, rec = itereate_dataset(train_dataset, model)
        print(f'Train Loss: {loss / len(train_dataset)}')
        print(f'Train Prec: {prec / len(train_dataset)}')
        print(f'Train Rec: {rec / len(train_dataset)}')
    if val_dataset != None:
        loss, prec, rec = itereate_dataset(val_dataset, model)
        print(f'Val Loss: {loss / len(val_dataset)}')
        print(f'Val Prec: {prec / len(val_dataset)}')
        print(f'Val Rec: {rec / len(val_dataset)}')

In [33]:
# Gether datasets
batch_size = 2
train_dataset = DolphinDataset(train, encoder, image_size=(1024, 1024))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
val_dataset = DolphinDataset(val, encoder, image_size=(1024, 1024))
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
test_dataset = DolphinDataset(test, encoder, image_size=(1024, 1024))
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)

In [27]:
model = get_model('../../ml_research_data/dolphin_weights/torch_with_momentum/49.pth')
eval_model(model, train_dataset=None, val_dataset=val_dataloader)

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 253/253 [00:34<00:00,  7.44it/s]

Val Loss: 0.8786258164387409
Val Prec: 0.04677207095055241
Val Rec: 0.046508566325478874


In [30]:
model = get_model('../../ml_research_data/dolphin_weights/torch_with_momentum/49.pth')
eval_model(model, train_dataset=None, val_dataset=test_dataloader)

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 256/256 [00:34<00:00,  7.33it/s]

Val Loss: 0.8913580208686653
Val Prec: 0.04531250236323103
Val Rec: 0.045182294023106806


In [35]:
model = get_model('../../ml_research_data/dolphin_weights/torch_with_momentum/49.pth')
eval_model(model, train_dataset=None, val_dataset=train_dataloader)

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 25008/25008 [52:44<00:00,  7.90it/s]

Val Loss: 0.8608732811835452
Val Prec: 0.04648845945164664
Val Rec: 0.04610125000513518
